**Instalando o GROQ**

In [1]:
%pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 2.9 MB/s eta 0:00:00


**Definição de uma variavel de ambiente da chave API**

No caso com o nome 'GROQ_API_KEY', junto com a chave que foi adiquirida.



In [2]:
import os
os.environ['GROQ_API_KEY'] = "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"

**Inicialização do cliente GROQ e teste da chave API**

In [3]:
import os
from groq import Groq

client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)

#Iniciamos o groq e aplicamos a chave api

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Explain the importance of fast language models",
        }
    ],
    model="llama-3.3-70b-versatile",
)

#Uma mensagem simples para testar se tudo está certo até agora

print(chat_completion.choices[0].message.content)

Fast language models are crucial in today's technology landscape, and their importance can be understood from several perspectives:

1. **Efficient Processing**: Fast language models can process and analyze large amounts of text data quickly, which is essential for applications that require real-time or near-real-time processing, such as chatbots, virtual assistants, and sentiment analysis tools.
2. **Improved User Experience**: Fast language models enable applications to respond rapidly to user input, providing a more seamless and interactive experience. This is particularly important for applications that require immediate feedback, such as language translation, text summarization, and question-answering systems.
3. **Scalability**: Fast language models can handle a large volume of requests and data, making them ideal for large-scale applications, such as social media platforms, search engines, and content management systems.
4. **Reduced Latency**: Fast language models minimize late

**Criação do Agente em si**

In [4]:
class Agent: #Criação da clase agente

  def __init__ (self,client,system): #definição de uma função que irá iniciar meu agente
    self.client = client #Inicializamos primeiro o cliente
    self.system = system #Logo em seguida iremos inicializar o "System Prompt"
    self.messages = [] #Criação de uma lista de um loop de mensagens que o agente irá incluir em sua memória
    if self.system is not None: #Caso exista uma mensagem do sistema nos acidionaremos à nossa lista de mensagens
      self.messages.append({"role": "system", "content": self.system})

  def __call__ (self, message = ""): #Uma chamada que será executada sem que o Agent for executado
    if message: #Se o usuária enviar uma mensagem ela será adicionada na lista
      self.messages.append({"role": "user", "content": message})
    result = self.execute() #Agora executaremos o modulo de linguagem, ou seja o Agent
    self.messages.append({"role": "assistant", "content": result}) #Adicionaremos o resultado do "def execute" à lista
    return result

  def execute(self): #Irá rodar todas as mensagens do usuária, junto ao modelo desejado
    completion = client.chat.completions.create(
        messages= self.messages,
        model="llama-3.3-70b-versatile",
    )
    return completion.choices[0].message.content

**Crição do Prompt do sistema e das ferramente que ele terá a disposição**

O meu desejo para criar uma pratica com o agente que foi ensinado em video era que ele pudesse acessar informações de peso e altura de certas pessoas e com isso calcular o BMI (BMI é a expressão em inglês de IMC que significa Índice de Massa Corporal) esse era meu objetivo, porém após terminar eu achei que seria interessante se o agente também pudesse dizer qual o estado corporal do indivíuo escolhido, se ele está abaixo do peso, acima, obseidade e etc, assim eu criei esse prompt para que isso fosse possível.

In [5]:
#Prompt do sistema
system_prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

get_weight_data:
e.g. get_weight_data: Pedro
returns weight of the person in kilograms

get_height_data:
e.g. get_height_data: Pedro
returns height of the person in meters

calcule_IMC
e.g. calcule_BMI: 86.32/1.80**2
returns the BMI of a person

Example session:

Question: What is Pedro's BMI?

Thought: I need to find Pedro's weight first.
Action: get_weight_data: Pedro
PAUSE

You will be called again with this:


Observation: 86.2

Thought: Now I need to find Pedro's height.
Action: get_height_data: Pedro
PAUSE

You will be called again with this:


Observation: 1.80

Thought: Now I have both weight and height. I will now calculate the BMI using the correct formula and expect the result as my next observation.
Action: calculate_IMC: 86.2 / (1.80 ** 2)
PAUSE

You will be called again with this:

Observation: 26.60

Thought: Now I have Pedro's. I will cheky what is his body condition,
Action: classify_BMI: 26.60
PAUSE

You will be called again with this:

Observation: Overweight

If you have the answer, output it as the Answer.

Answer: Pedro's BMI is 26.60 and he is Overweight.

Now it's your turn:
""".strip()



# **TOOLS**

Aqui estão as tools relacionadas ao meu agente 'Nutricionista'

In [6]:
#Ferramentas
#tranfomaremos em um função, as ferramentas que foram mencionadas no prompt (calcular, e pegar a massa dos planetas) , para que assim o Agent possa ter acesso a ela

def calculate(operation) -> float:
  return eval(operation)

def calculate_BMI(operation) -> float:
  return eval(operation)

def get_weight_data(name: str) -> float:
    match name.lower():
        case "pedro":
            return 86.2
        case "caio":
            return 55.7
        case "vitor":
            return 85.0
        case "ana":
            return 62.5
        case "maria":
            return 72.0
        case "joao":
            return 79.0
        case "lucas":
            return 68.0
        case "carla":
            return 58.1
        case "patricia":
            return 70.0
        case "gabriel":
            return 77.5
        case "luciana":
            return 64.5
        case "rafael":
            return 82.5
        case "gabriela":
            return 60.5
        case "beatriz":
            return 65.0
        case _:
            return 0.0

def get_height_data(name: str) -> float:
    match name.lower():
        case "pedro":
            return 1.80
        case "caio":
            return 1.75
        case "vitor":
            return 1.85
        case "ana":
            return 1.65
        case "maria":
            return 1.70
        case "joao":
            return 1.78
        case "lucas":
            return 1.72
        case "carla":
            return 1.60
        case "patricia":
            return 1.68
        case "gabriel":
            return 1.80
        case "luciana":
            return 1.63
        case "rafael":
            return 1.82
        case "gabriela":
            return 1.67
        case "beatriz":
            return 1.72
        case _:
            return 0.0

def classify_BMI(BMI: float) -> str:
    # Garantir que BMI seja um número do tipo float
    BMI = float(BMI)

    match BMI:
        case BMI if BMI < 18.5:
            return "Underweight"
        case BMI if 18.5 <= BMI < 25:
            return "Normal Weight"
        case BMI if 25 <= BMI < 30:
            return "Overweight"
        case BMI if 30 <= BMI < 35:
            return "Obesity Grade 1"
        case BMI if 35 <= BMI < 40:
            return "Obesity Grade 2"
        case _:
            return "Obesity Grade 3"


**Crição de um loop para o Agent**

Isso irá acabar com a necessidade de uma pessoa ficar chamando o agente toda vez que desejarmos que ele pense (assim como feito anteriormente), através de um loop ele será capaz de fazer isso por conta própria

In [7]:
import re # Importação da biblioteca "Regular Expressions", pois precisaremos analisar os conteúdos de uma string

def agent_loop(max_iterations=10, query: str = ""):
    # Inicialização do agente
    agent = Agent(client, system_prompt)
    tools = ['calculate', 'calculate_BMI', 'get_weight_data', 'get_height_data', 'classify_BMI'] # Inicializando o nome das ferramentas que temos
    next_prompt = query
    i = 0

    while i < max_iterations: # Início do loop
        i += 1
        result = agent(next_prompt) # Chamamos o Agent e verificamos o resultado
        print(result)

        if "PAUSE" in result and "Action" in result: # Se o resultado tiver uma ação e pausa
            action = re.findall(r"Action: ([a-z_]+): (.+)", result, re.IGNORECASE)

            if action:
                chosen_tool = action[0][0]
                arg = action[0][1]

                if chosen_tool in tools:
                    result_tool = eval(f"{chosen_tool}('{arg}')")
                    next_prompt = f"Observation: {result_tool}"

                else:
                    next_prompt = "Observation: Tool not found"
            else:
                next_prompt = "Observation: No valid action found"

            print(next_prompt)
            continue

        if "Answer" in result: # A resposta foi encontrada e podemos sair do loop
            break


# **Testes**
Meu principal intuito, como mencionado anteriormente era encontrar o BMI da uma pessoa e descobrir qual era o estado corporal da quela pessoa, sobrepeso, obeso ou etc, assim eu fiz o teste e ocorreu tudo certo.

In [8]:
agent_loop(query="what is Vitor's BMI and body condition?")

Thought: I need to find Vitor's weight first to calculate his BMI.
Action: get_weight_data: Vitor
PAUSE
Observation: 85.0
Thought: Now I need to find Vitor's height to calculate his BMI using the correct formula.
Action: get_height_data: Vitor
PAUSE
Observation: 1.85
Thought: Now I have both weight and height. I will now calculate the BMI using the correct formula and expect the result as my next observation.
Action: calcule_IMC: 85.0 / (1.85 ** 2)
PAUSE
Observation: Tool not found
Thought: It seems the tool "calcule_IMC" is not available. However, I can use the "calculate" tool to calculate the BMI. The formula for BMI is weight / height^2.
Action: calculate: 85.0 / (1.85 ** 2)
PAUSE
Observation: 24.835646457268076
Thought: Now I have Vitor's BMI. I will now classify his BMI to determine his body condition.
Action: classify_BMI: 24.835646457268076
PAUSE
Observation: Normal Weight
Thought: Now I have Vitor's BMI and his body condition. I can now provide the answer.
Action: None
Answer:

Nesse segundo texte eu queria testar além, minha intenção era ver se o agent realmente poderia responder outras perguntas apenas com as ferramentas que eu lhe dei, e não apenas à pergunta a qual ele foi intruido a pensar, agir e observar, teste esse que deu certo.

In [9]:
agent_loop(query="Between Pedro and Vitor who has the highest BMI?")

Thought: To determine who has the highest BMI between Pedro and Vitor, I need to calculate the BMI of both individuals. First, I need to find Pedro's weight and height, then Vitor's weight and height, and finally calculate their respective BMIs.

Action: get_weight_data: Pedro
PAUSE
Observation: 86.2
Thought: Now that I have Pedro's weight, I need to find his height to calculate his BMI.

Action: get_height_data: Pedro
PAUSE
Observation: 1.8
Thought: Now that I have Pedro's weight and height, I can calculate his BMI. However, to compare it with Vitor's, I also need Vitor's weight and height. Let's calculate Pedro's BMI first.

Action: calculate: 86.2 / (1.8 ** 2)
PAUSE
Observation: 26.604938271604937
Thought: Now that I have Pedro's BMI, I need to find Vitor's weight to calculate his BMI for comparison.

Action: get_weight_data: Vitor
PAUSE
Observation: 85.0
Thought: Now that I have Vitor's weight, I need to find his height to calculate his BMI.

Action: get_height_data: Vitor
PAUSE
Ob